<a href="https://colab.research.google.com/github/IFuentesSR/Reservoir_EMS/blob/main/Reservoir_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Google Earth Engine Library and authenticate

In [1]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=ZwdNK2ZJUKayBSMjS-qRb7jfjIi70rI56Aj7WvvgaSY&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g5hIS8AmCmDJ-5YgLxbDI0XwXzXnsTpI_lYSyKW47Erukp-OeIMHEQ

Successfully saved authorization token.


## Importing folium and other required libraries

In [2]:
import folium
import time


print(folium.__version__)

0.8.3


## Defining some pre-processing function

In [3]:
def generateGrid(xmin, ymin, xmax, ymax, dx, dy, marginx, marginy):
  xx = ee.List.sequence(xmin, ee.Number(xmax).subtract(ee.Number(dx).multiply(0.9)), dx)
  yy = ee.List.sequence(ymin, ee.Number(ymax).subtract(ee.Number(dy).multiply(0.9)), dy)
  
  def innerX(x):
    def innerY(y):
      x1 = ee.Number(x).subtract(marginx)
      x2 = ee.Number(x).add(ee.Number(dx)).add(marginx)
      y1 = ee.Number(y).subtract(marginy)
      y2 = ee.Number(y).add(ee.Number(dy)).add(marginy)
      coords = ee.List([x1, y1, x2, y2])
      rect = ee.Algorithms.GeometryConstructors.Rectangle(coords, 'EPSG:4326', False)
      return ee.Feature(rect)
    return yy.map(innerY)
  cells = xx.map(innerX).flatten()

  return ee.FeatureCollection(cells)


def parse_id(fea):
  return fea.set('id', ee.Number.parse(fea.id()))


def CloudMaskS2(image):
   prop = image.propertyNames()
   QA = image.select('SCL')
   clouds = 9
   cirrus = 8
   thin_cirrus = 10
   shadows = 3
   mask = QA.neq(clouds).And(QA.neq(shadows)).And(QA.neq(thin_cirrus)) # .And(QA.neq(cirrus))
   return image.updateMask(mask).multiply(0.0001).copyProperties(image, prop)


# https://www.sciencedirect.com/science/article/pii/S0034425715302753?via%3Dihub
def WI(img):
  return img.expression('1.7204 + 171 * Green + 3 * Red - 70 * NIR - 45 * SWIR1 - 71 * SWIR2',
  {'Green':img.select('B3'),'Red':img.select('B4'), 
  'NIR':img.select('B8'),'SWIR1':img.select('B11'),
  'SWIR2':img.select('B12')}).rename('WI').copyProperties(img, ['system:time_start'])


def hillShadow(img):
  hs = ee.Terrain.hillShadow(srtm,
                             img.get('MEAN_SOLAR_AZIMUTH_ANGLE'),
                             img.get('MEAN_SOLAR_ZENITH_ANGLE'),
                             100,
                             True)
  return img.updateMask(hs)


def water_mask(img):
  return img.gt(0).copyProperties(img, ['system:time_start'])


def pixel_mask(img):
  return img.gte(0)


def getData(i):
  return i.setMulti({'area': i.geometry().area(1),
                     'peri': i.geometry().perimeter(1),
                     'ratio': ee.Number(i.geometry().area(1)).divide(i.geometry().perimeter(1))
  })


  


## Calling datasets to use in a first instance

In [4]:
Namoi = ee.FeatureCollection('users/ignisfausto/Namoi')

grid = generateGrid(147.2, -32, 151.80, -29.7, 1.5, 1.2, 0, 0)
grid = grid.map(parse_id)
grid = grid.filterBounds(Namoi)

hands100 = ee.ImageCollection('users/gena/global-hand/hand-100')
hands100 = hands100.mosaic()

SR = ee.ImageCollection("COPERNICUS/S2_SR")
srtm = ee.Image("USGS/SRTMGL1_003")
river = ee.FeatureCollection('users/ignisfausto/MajorHydro')
major = ee.FeatureCollection('users/ignisfausto/NamoiReservoirs')

LC = ee.ImageCollection("MODIS/006/MCD12Q1")
LC = LC.select('LC_Type1').filterDate('2018-01-01', '2020-01-01').first()


## Mapping the catchment and the grid cells for data exportation.

The grid is used because running the vectorization of water bodies for the entire catchment results in memory and computation time out errors

In [5]:
mapid = ee.FeatureCollection(grid).getMapId({'opacity':0.1})
mapid2 = Namoi.getMapId()
map = folium.Map(location=[-30.63, 149.91], zoom_start=8)
folium.TileLayer(
    tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='satellite',
  ).add_to(map)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='grid',
  ).add_to(map)
folium.TileLayer(
    tiles=mapid2['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Namoi',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

## Definition of parameters for water detection
Selection of start and end date to filter the image collection, and preprocessing of images to conver them into a water mask collection

In [6]:
start = '2019-01-01'
end = '2020-01-01'

ids = grid.aggregate_array('id')
ids = ids.getInfo()

# SRfiltered = SR.filterBounds(Namoi).filterDate(start, end).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
# SRfiltered = SRfiltered
# SRfiltered = SRfiltered.map(CloudMaskS2)#.map(hillShadow)
# waterSR = SRfiltered.map(WI).map(water_mask)


In [7]:
print(SR.filterBounds(Namoi).filterDate(start, end).size().getInfo())
print(SR.filterBounds(Namoi).filterDate(start, end).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)).size().getInfo())

1514
935


## **Alternatively**

if more images are needed, since we are filtering the Sentinel 2 SR collection to get only the images with less than 5% clouds, the probability of clouds for sentinel 2 images dataset can be merged into the SR dataset.

This will lead to some misclassification errors (due to shadows or remanent clouds), but their occurrence is limited, so by setting the occurrence probability to a threshold higher than 0.1 we make sure that we are picking up only surface water and not misclassifier water.

The code below is used **only** if more images are needed and replaces the SR variable, otherwise skip this cell.

In [8]:
def getS2_SR_CLOUD_PROBABILITY():
  def mergeImageBands(joinResult):
    return ee.Image(joinResult.get('primary')).addBands(joinResult.get('secondary'))
  innerJoined = ee.Join.inner().apply(primary=ee.ImageCollection("COPERNICUS/S2_SR"),
                                    secondary=ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY"),
                                    condition=ee.Filter.equals(leftField='system:index',
                                                               rightField='system:index'))

  newCollection = innerJoined.map(mergeImageBands);
  return ee.ImageCollection(newCollection)


def maskClouds(img):
  cloudProbabilityThreshold = 30
  cloudMask = img.select('probability').lt(cloudProbabilityThreshold)
  return img.updateMask(cloudMask)


def shadowMask(img):
   QA = img.select('SCL')
   shadows = 3
   mask = QA.neq(shadows)
   return img.updateMask(mask).multiply(0.0001)


SR = getS2_SR_CLOUD_PROBABILITY()
SRfiltered = SR.filterBounds(Namoi).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
SRfiltered = SRfiltered.filterDate(start, end)
SRfiltered = SRfiltered.map(maskClouds).map(shadowMask)
waterSR = SRfiltered.map(WI).map(water_mask)

## Water Occurrence Probability

Water occurrence probability was calculated as:

$P_{(i, j)} = \frac{1}{n} \sum_{i=1}^{n}W_{(i, j)}$

where $P_{(i, j)}$ is the probability of water occurrence at the pixel coordintes $i, j$, n is the times every pixel takes place in the collection and $W_{(i, j)}$ is the times that water is found in the pixel of coordinates $i, j$.

Additionally, hand values over 50 where assumed as not being flooded.


In [9]:
pixels = ee.Image(waterSR.map(pixel_mask).sum())
waterOccurrence = waterSR.sum().divide(waterSR.map(pixel_mask).sum()).clip(Namoi)
waterOccurrence = waterOccurrence.where(hands100.gt(50), 0)
waterOccurrence1 = waterOccurrence.gt(0.2)
WO = waterOccurrence1.updateMask(waterOccurrence1).addBands(waterOccurrence)


## Mapping water occurrence in the catchment

In [10]:
# waterOccurrence = waterOccurrence.selfMask()
mapidOcc = waterOccurrence.getMapId({'min': 0, 'max': 1, 'palette':'FFFFFF, 87CEFA, 00BFFF, 1E90FF, 4169E1, 00008B'})
mapidCount = pixels.getMapId({'min': 20, 'max': 60, 'palette': 'FF0000, 00FF00', 'opacity':0.7})
mapidLC = LC.clip(Namoi).getMapId({'min':1, 'max':17, 'palette': [
    '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159',
    'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c',
    '69fff8', 'f9ffa4', '1c0dff'
  ]})
map = folium.Map(location=[-30.63, 149.91], zoom_start=9)
folium.TileLayer(
    tiles=mapid2['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Namoi',
  ).add_to(map)
folium.TileLayer(
    tiles=mapidLC['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Land cover',
  ).add_to(map)
folium.TileLayer(
    tiles=mapidCount['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Times',
  ).add_to(map)
folium.TileLayer(
    tiles=mapidOcc['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Occurrence',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [ ]:
ee.batch.Export.image.toDrive(image=waterOccurrence,
                              region=Namoi.geometry(),
                              scale=10,
                              description='Occurrence',
                              maxPixels=1e13).start()

## Selection of barren areas
Bare land areas were selected from the land cover map in order to filter reservoirs contaned in such areas, which most of the time are related to mining operations

In [11]:
barren = LC.eq(16)
barren = barren.updateMask(barren).addBands(barren)
barren = barren.reduceToVectors('first', Namoi, 500)

### water reservoirs were delineated assuming that these are present in at least 20% of the images (occurrence > 0.2).

Vectorization was carried out in each cell of the grid due to constraints in the computational capacity.

Reservoirs are exported for every cell in the grid.

**Run this cell only** if you want to export reservoirs into your assets, **otherwise skip this cell** (it takes at least 50 minutes runing because in each step of the loop a 10' delayer is especified.



In [ ]:
'''Just run this cell if you want to save the data in your assets''' 

### Replace here with your account path 
assetId = 'users/ignaciofuentessanroman/Reservoirs' 
for n in ids:
  res = WO.reduceToVectors(reducer=ee.Reducer.mean(),
                            geometry=grid.filter(ee.Filter.eq('id', n)).first().geometry(),
                            scale=10,
                            maxPixels=1e13)
  task = ee.batch.Export.table.toAsset(collection=res,
                                      description='exam1',
                                      assetId='{}/reservoirsNamoi{}'.format(assetId,
                                                                            n))
  task.start()
  time.sleep(10*60)

Otherwise, just use these (I have shared them). Water vectors were stored as assets in Reservoirs folder.

In [12]:
table = ee.FeatureCollection("users/ignaciofuentessanroman/Reservoirs/reservoirsNamoi1")
table2 = ee.FeatureCollection("users/ignaciofuentessanroman/Reservoirs/reservoirsNamoi2")
table3 = ee.FeatureCollection("users/ignaciofuentessanroman/Reservoirs/reservoirsNamoi3")
table4 = ee.FeatureCollection("users/ignaciofuentessanroman/Reservoirs/reservoirsNamoi4")
table5 = ee.FeatureCollection("users/ignaciofuentessanroman/Reservoirs/reservoirsNamoi5")
res = table.merge(table2).merge(table3).merge(table4).merge(table5)

## Filtering reservoirs

Different conditions may be passed for the filtering and detection of water reservoirs.


*   area > 10,000 m²
*   ratio area-perimeter > 25
*   area < 500,000 m²
*   reservoirs not intersecting major rivers
*   reservoirs not intersecting major reservoirs
*   reservoirs not contained in barren areas (mines, some)
*   reservors below 410 m



In [13]:
res = res.map(getData)
res = res.filter(ee.Filter.gt('area', 10000))
res = res.filter(ee.Filter.gt('ratio', 25))
res = res.filter(ee.Filter.lt('area', 500000))
res = res.filter(ee.Filter.intersects('.geo', river.geometry()).Not())
res = res.filter(ee.Filter.intersects('.geo', major.geometry()).Not())
res = res.filter(ee.Filter.intersects('.geo', barren.geometry()).Not())
res = srtm.reduceRegions(res, 'mean', 30)
res = res.filter(ee.Filter.lt('mean', 410))

# Map of farm reservoirs

Move around to see areas with reservoirs.
You can turn off/on the satellite image and the reservoirs layers.
Consider that the satellite image is not taken necesarily at the sam year than the detection, and that is a snapshot at the specific date, while the detection assumes a water occurrence greater than 0.2 in the collection.

In [14]:
mapidRes = ee.FeatureCollection(res).getMapId()
map = folium.Map(location=[-30.2, 149.6], zoom_start=11)
folium.TileLayer(
    tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='satellite',
  ).add_to(map)
folium.TileLayer(
    tiles=mapidRes['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='reservoirs',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [ ]:
Bathy_reservoirs = ee.FeatureCollection('users/ignisfausto/contourReservoirs')